\section*{Методы решения ОДУ}

\subsection*{Постановка задач}
1. Построить экстраполяционный метод Адамса пятого порядка (дать представление в двух формах).
2. Определить порядок точности метода
$$\begin{cases}
y_{j+1}=y_j+\frac{\tau}{6}(f_j+4f_{j+1\frac{1}{2}}+f_{j+1}),\\
y_j=y_{j+1}-\tau f_{j+\frac{1}{2}}, \\
y_{j+\frac{1}{2}}=y_{j+1}-\frac{\tau}{2}f_{j+1}.
\end{cases}$$
3. Найти интервал устойчивости метода из п. 1;
4. С заданной точность $\epsilon=10^{-4}$ найти решение задачи Коши с помощью: а) метода последовательного повышения порядка точности четвертого порядка; б) явного метода Рунге-Кутта третьего порядка; в) интерполяционого метода Адамса третьнго порядка
$$\begin{cases}
u'=\frac{1}{5}\begin{pmatrix}-104 & -198 \\
-198 & -401 \end{pmatrix} u, t \in [0; 5], \\
u(0)=\begin{pmatrix}5 \\ 10 \end{pmatrix}.
\end{cases}$$